In [157]:
import pandas as pd
import numpy as np
import pm4py
import os
from sklearn.neighbors import KernelDensity
import random
from collections import Counter
import copy
import warnings
warnings.filterwarnings("ignore")

In [158]:
def csv_to_xes(file, add_startend, id_column, act_column, start_time_column, end_time_column, resource_column):
    import_file = f'{file}.csv' #
    df = pd.read_csv(import_file)
#     print(df)
    id_column = id_column
    act_column = act_column
    start_time_column = start_time_column
    end_time_column = end_time_column
    resource_column = resource_column
    state_column = 'lifecycle:transition'
    time_column = 'time:timestamp'
    df[resource_column].fillna('no_resource', inplace=True)
    df.to_csv(f'C:/Users/19wuj/PycharmProjects/CoDi/tabular_datasets/diffu/PurchasingExample_0.2/{file}.csv', index=False)
    if add_startend:
        # pre-processing
        mydata = pd.DataFrame()
        for key, group in df.groupby(id_column):
            group = add_start_end(df, group, act_column, resource_column, start_time_column, end_time_column)   # add start event in the beginning and end in the last 
            mydata = pd.concat([mydata, group], ignore_index=True)
        df = mydata

    
    df_start = df[[id_column, act_column, start_time_column, resource_column]].rename(columns={start_time_column:time_column})
    df_start[state_column] = 'start'
    df_start

    df_end = df[[id_column, act_column, end_time_column, resource_column]].rename(columns={end_time_column:time_column})
    df_end[state_column] = 'complete'

    df_start_end = pd.concat([df_start, df_end])
    df_start_end['index'] = df_start_end.index
    df_start_end[time_column] = pd.to_datetime(df_start_end[time_column])
    new = pd.DataFrame()
    for key, group in df_start_end.groupby(id_column):
        temp = group.sort_values(by=[time_column, 'index'], ascending=[True, True]).reset_index(drop=True)
        new = pd.concat([new,temp],ignore_index = True)
    del new['index']
    df = new
    
    
#     print(new.columns)
    df['concept:name'] = df[act_column].astype('str')
    df[id_column] = df[id_column].astype('str')
    print(df)
    df = df[[id_column, 'concept:name', state_column, resource_column, time_column, act_column]]
#     df = df[[id_column, act_column,  state_column, resource_column, time_column]]
    df = df.rename(columns = { resource_column: 'user', act_column: 'task'})
    
    pm4py.write.write_xes(df, f'C:/Users/19wuj/PycharmProjects/CoDi/tabular_datasets/diffu/PurchasingExample_0.2/{file}.xes', case_id_key=id_column)

In [5]:
csv_to_xes('train_PurchasingExample', add_startend = False, id_column='caseid', act_column='task', start_time_column='start_timestamp', end_time_column='end_timestamp', resource_column='user')

      caseid                          task      time:timestamp  \
0          2                         Start 2011-01-01 00:16:00   
1          2                         Start 2011-01-01 00:16:00   
2          2   Create Purchase Requisition 2011-01-01 00:16:00   
3          2   Create Purchase Requisition 2011-01-01 00:29:00   
4          2  Create Request for Quotation 2011-01-01 08:16:00   
...      ...                           ...                 ...   
16373   1949   Create Purchase Requisition 2011-10-13 07:21:00   
16374   1949  Analyze Purchase Requisition 2011-10-13 17:03:00   
16375   1949  Analyze Purchase Requisition 2011-10-13 17:09:00   
16376   1949                           End 2011-10-13 17:09:00   
16377   1949                           End 2011-10-13 17:09:00   

                      user lifecycle:transition                  concept:name  
0                    Start                start                         Start  
1                    Start             complete

exporting log, completed traces ::   0%|          | 0/486 [00:00<?, ?it/s]

In [159]:
# info of train_data
if_csv = False #
import_file = f'C:/Users/19wuj/PycharmProjects/CoDi/tabular_datasets/diffu/PurchasingExample_0.2/train_PurchasingExample.xes' #
id_column = 'caseid'
act_column = 'concept:name'
time_column = 'time:timestamp'
resource_column = 'user'
state_column = 'lifecycle:transition'

dirStr, ext = os.path.splitext(import_file)
file_name = dirStr.split("/")[-1]


data = pm4py.read.read_xes(import_file)

df = copy.deepcopy(pd.DataFrame(data,columns=[id_column, act_column, time_column, resource_column, state_column]))

parsing log, completed traces ::   0%|          | 0/486 [00:00<?, ?it/s]

In [160]:
df['waiting_time'] = None
df['process_time'] = None
df['last_complete_event'] = None
df['preceding_evts'] = None
df['paired_event'] = None
df['next'] = None
df['index'] = df.index
for key, group in df.groupby(id_column):
    flag = 0
    i = list(group.index)[0]
    preceding_evt = []
    not_complete_evt_idx = []
    j = 0
    df['waiting_time'].loc[i] = 0
    not_complete_evt_idx.append(i)
    i+=1
    while j<len(group)-1:
        j+=1
        cur_act = df.loc[i]
        if cur_act[state_column] == 'complete':
            flag = 0
            preceding_evt.append(cur_act.name)
            last_complete_evt_idx = i
            
            for each_idx in not_complete_evt_idx:
                to_pair = df.loc[each_idx]
                if (cur_act[act_column] == to_pair[act_column]) \
                and (cur_act[resource_column] == to_pair[resource_column]):
                    df['paired_event'].loc[i] = each_idx
                    not_complete_evt_idx.remove(each_idx)
                    df['process_time'].loc[i] = (df[time_column].loc[i] - df[time_column].loc[each_idx]).total_seconds()
                    df['last_complete_event'].loc[i] = df['last_complete_event'].loc[each_idx]
                    break
            
        else:
            if flag == 1:
                df['preceding_evts'].loc[i] = df['preceding_evts'].loc[i-1]
            else:
                df['preceding_evts'].loc[i] = preceding_evt#tuple(sorted, reverse=False))
            for each in df['preceding_evts'].loc[i]:
                if df['next'].loc[each] == None:
                    df['next'].loc[each] = [i]
                else:
                    temp = df['next'].loc[each]
                    temp.append(i)
                    df['next'].loc[each] = temp
            flag = 1
            preceding_evt = []
            not_complete_evt_idx.append(i)
            
            df['last_complete_event'].loc[i] = last_complete_evt_idx
            df['waiting_time'].loc[i] = (df[time_column].loc[i] - df[time_column].loc[last_complete_evt_idx]).total_seconds()
        i+=1

In [161]:
def fillna_parallel_data(bb):
    i = bb.index[0]
    j = 0 
    while j < len(bb):
        if bb[state_column].loc[i] == 'complete':
            paired = int(bb['paired_event'].loc[i])
            bb['process_time'].loc[paired] = bb['process_time'].loc[i] 
            bb['paired_event'].loc[paired] = i
            bb['next'].loc[paired] = bb['next'].loc[i]
            bb['waiting_time'].loc[i] = bb['waiting_time'].loc[paired] 
            bb['preceding_evts'].loc[i] = bb['preceding_evts'].loc[paired]
            bb['next'].loc[paired] = bb['next'].loc[i] 
        i+=1
        j+=1
fillna_parallel_data(df)


In [163]:
df

,caseid,concept:name,time:timestamp,user,lifecycle:transition,waiting_time,process_time,last_complete_event,preceding_evts,paired_event,next,index
0,2,Start,2011-01-01 00:16:00+00:00,Start,start,0,0,None,None,1,[2],0
1,2,Start,2011-01-01 00:16:00+00:00,Start,complete,0,0,None,None,0,[2],1
2,2,Create Purchase Requisition,2011-01-01 00:16:00+00:00,Immanuel Karagianni,start,0,780,1,[1],3,[4],2
3,2,Create Purchase Requisition,2011-01-01 00:29:00+00:00,Immanuel Karagianni,complete,0,780,1,[1],2,[4],3
4,2,Create Request for Quotation,2011-01-01 08:16:00+00:00,Alberto Duport,start,28020,600,3,[3],5,[6],4
...,...,...,...,...,...,...,...,...,...,...,...,...
16373,1949,Create Purchase Requisition,2011-10-13 07:21:00+00:00,Tesca Lobes,complete,0,3240,16371,[16371],16372,[16374],16373
16374,1949,Analyze Purchase Requisition,2011-10-13 17:03:00+00:00,Francis Odell,start,34920,360,16373,[16373],16375,[16376],16374
16375,1949,Analyze Purchase Requisition,2011-10-13 17:09:00+00:00,Francis Odell,complete,34920,360,16373,[16373],16374,[16376],16375
16376,1949,End,2011-10-13 17:09:00+00:00,End,start,0,0,16375,[16375],16377,None,16376


In [164]:
df = df[df[state_column]=='start']

# df = df.rename(columns = {'concept:name':'activity'})
file = df[[id_column,act_column, resource_column, 'waiting_time', 'process_time']]
file = file[file[act_column] != 'Start']
file = file[file[act_column] != 'End']
file = file.reset_index(drop=True)

file[act_column] = file[act_column].map(lambda x: '_'.join(x.split()))
file[resource_column] = file[resource_column].map(lambda x: '_'.join(x.split()))

file

,caseid,concept:name,user,waiting_time,process_time
0,2,Create_Purchase_Requisition,Immanuel_Karagianni,0,780
1,2,Create_Request_for_Quotation,Alberto_Duport,28020,600
2,2,Analyze_Request_for_Quotation,Karel_de_Groot,4080,240
3,2,Amend_Request_for_Quotation,Christian_Francois,2280,300
4,2,Analyze_Request_for_Quotation,Magdalena_Predutta,3240,1980
...,...,...,...,...,...
7212,1941,Analyze_Purchase_Requisition,Heinz_Gutschmidt,86160,240
7213,1941,Create_Request_for_Quotation,Francis_Odell,240,180
7214,1941,Analyze_Request_for_Quotation,Karel_de_Groot,154560,780
7215,1949,Create_Purchase_Requisition,Tesca_Lobes,0,3240


In [165]:
import math
file['waiting_time'] = file['waiting_time'].map(lambda x: math.log(x+1))
file['process_time'] = file['process_time'].map(lambda x: math.log(x+1))
file

,caseid,concept:name,user,waiting_time,process_time
0,2,Create_Purchase_Requisition,Immanuel_Karagianni,0.000000,6.660575
1,2,Create_Request_for_Quotation,Alberto_Duport,10.240710,6.398595
2,2,Analyze_Request_for_Quotation,Karel_de_Groot,8.314097,5.484797
3,2,Amend_Request_for_Quotation,Christian_Francois,7.732369,5.707110
4,2,Analyze_Request_for_Quotation,Magdalena_Predutta,8.083637,7.591357
...,...,...,...,...,...
7212,1941,Analyze_Purchase_Requisition,Heinz_Gutschmidt,11.363973,5.484797
7213,1941,Create_Request_for_Quotation,Francis_Odell,5.484797,5.198497
7214,1941,Analyze_Request_for_Quotation,Karel_de_Groot,11.948344,6.660575
7215,1949,Create_Purchase_Requisition,Tesca_Lobes,0.000000,8.083637


In [166]:
from collections import Counter
from sklearn import preprocessing
new_data = pd.DataFrame()
def get_info(df, new_df, column_name, column_type):
    if column_name == id_column:
        new_df[column_name] = df[column_name]
    else:  
        if column_type == 'categorical':
            df[column_name].fillna(f'NO_{column_name}',inplace=True)
            dict_ = {}
            a = Counter(df[column_name])     
            dict_['name'] = column_name
            dict_['size'] = len(a.keys())
            dict_['type'] = column_type
            enc=preprocessing.LabelEncoder()
            enc=enc.fit(list(a.keys()))#训练LabelEncoder,将电脑，手表，手机编码为0,1,2
            data=enc.transform(df[column_name])#使用训练好的LabelEncoder对原数据进行编码，也叫归一化
            dict_['i2s'] = list(enc.classes_)
            new_df[column_name] = data
            return dict_
        else:
            dict_ = {}
            dict_['max'] = max(df[column_name])
            dict_['min'] = min(df[column_name])
            dict_['name'] = column_name
            dict_['type'] = column_type
            new_df[column_name] = df[column_name]
            return dict_


In [167]:
file['user_2'] = file['user']
file

,caseid,concept:name,user,waiting_time,process_time,user_2
0,2,Create_Purchase_Requisition,Immanuel_Karagianni,0.000000,6.660575,Immanuel_Karagianni
1,2,Create_Request_for_Quotation,Alberto_Duport,10.240710,6.398595,Alberto_Duport
2,2,Analyze_Request_for_Quotation,Karel_de_Groot,8.314097,5.484797,Karel_de_Groot
3,2,Amend_Request_for_Quotation,Christian_Francois,7.732369,5.707110,Christian_Francois
4,2,Analyze_Request_for_Quotation,Magdalena_Predutta,8.083637,7.591357,Magdalena_Predutta
...,...,...,...,...,...,...
7212,1941,Analyze_Purchase_Requisition,Heinz_Gutschmidt,11.363973,5.484797,Heinz_Gutschmidt
7213,1941,Create_Request_for_Quotation,Francis_Odell,5.484797,5.198497,Francis_Odell
7214,1941,Analyze_Request_for_Quotation,Karel_de_Groot,11.948344,6.660575,Karel_de_Groot
7215,1949,Create_Purchase_Requisition,Tesca_Lobes,0.000000,8.083637,Tesca_Lobes


In [59]:
# new_data = pd.DataFrame()
# a = [act_column, 'user_2', resource_column]
# b = ['waiting_time', 'process_time']
# c = [act_column, resource_column]

# json_input = {}
# get_info(file, new_data, id_column, 'categorical')
# json_input['attention'] = [get_info(file, new_data, each, 'categorical') for each in c]
# json_input['columns'] = [get_info(file, new_data, each, 'categorical') for each in a]
# json_input['columns'].extend([get_info(file, new_data, each, 'continuous') for each in b])
# json_input['problem_type'] = 'no'

In [168]:
new_data = pd.DataFrame()
a = [act_column,  resource_column]
b = ['waiting_time', 'process_time']
c = [act_column, resource_column]

json_input = {}
get_info(file, new_data, id_column, 'categorical')
json_input['attention'] = [get_info(file, new_data, each, 'categorical') for each in c]
json_input['columns'] = [get_info(file, new_data, each, 'categorical') for each in a]
json_input['columns'].extend([get_info(file, new_data, each, 'continuous') for each in b])
json_input['problem_type'] = 'no'

In [50]:
for i in range(1):
    print(i)

0


In [12]:
# new_data['prev_acts'] = None
# new_data['prev_res'] = None
# new_data['curr_act'] = None
# data_add_prev = pd.DataFrame()
# for key, group in new_data.groupby(id_column):
#     for i in range(len(group)):
#         prev_acts = list(group[act_column].iloc[0:i].astype('str'))
#         prev_acts_str = ' '.join(prev_acts)
#         group['prev_acts'].iloc[i] = prev_acts_str
        
#         prev_res = list(group[resource_column].iloc[0:i].astype('str'))
#         prev_res_str = ' '.join(prev_res)
#         group['prev_res'].iloc[i] = prev_res_str
        
#         group['curr_act'].iloc[i] = group[act_column].iloc[i].astype('str')
#     data_add_prev = pd.concat([data_add_prev, group])

In [169]:
new_data['prev_acts'] = None
new_data['prev_res'] = None
new_data['curr_act'] = None
data_add_prev = pd.DataFrame()
for key, group in new_data.groupby(id_column):
    for i in range(len(group)):
#         if i != 0:
        prev_acts = list(group[act_column].iloc[0:i])
#         prev_acts_str = ' '.join(prev_acts)
        group['prev_acts'].iloc[i] = prev_acts

        prev_res = list(group[resource_column].iloc[0:i])
#         prev_res_str = ' '.join(prev_res)
        group['prev_res'].iloc[i] = prev_res
        #print(group)
        group['curr_act'].iloc[i] = group[act_column].iloc[i]
    data_add_prev = pd.concat([data_add_prev, group])

In [170]:
data_add_prev['curr_act'] = data_add_prev['curr_act'].map(lambda x: [x])
data_add_prev

,caseid,concept:name,user,waiting_time,process_time,prev_acts,prev_res,curr_act
80,10,10,7,0.000000,8.026497,[],[],[10]
81,10,12,13,10.090838,6.042633,[10],[7],[12]
82,10,4,11,9.798183,7.390799,"[10, 12]","[7, 13]",[4]
83,10,18,15,11.226056,6.928538,"[10, 12, 4]","[7, 13, 11]",[18]
84,10,11,15,7.273093,8.137688,"[10, 12, 4, 18]","[7, 13, 11, 15]",[11]
...,...,...,...,...,...,...,...,...
5913,993,2,10,11.305914,5.707110,[10],[4],[2]
5914,997,10,17,0.000000,7.878913,[],[],[10]
5915,997,2,20,10.525702,5.484797,[10],[17],[2]
5916,997,12,20,5.198497,5.198497,"[10, 2]","[17, 20]",[12]


In [171]:
def padding(list_, pad_number, length):
    len_list = len(list_) 
    if len_list < length:
        list_ = list_+ (length - len_list)*[pad_number]
    return list_

def padding_key(list_,act_pad):
    return [True if each == act_pad else False for each in list_]

In [172]:
len_act = max(data_add_prev['prev_acts'].map(lambda x: len(x)))
len_res = max(data_add_prev['prev_res'].map(lambda x: len(x)))
act_pad = json_input['attention'][0]['size']
res_pad = json_input['attention'][1]['size']



data_add_prev['prev_acts'] = data_add_prev['prev_acts'].map(lambda x: padding(x,act_pad,len_act))
data_add_prev['prev_acts_key_padding'] = data_add_prev['prev_acts'].map(lambda x: padding_key(x,act_pad))
data_add_prev['prev_res'] = data_add_prev['prev_res'].map(lambda x: padding(x,res_pad,len_res))
data_add_prev['prev_res_key_padding'] = data_add_prev['prev_res'].map(lambda x: padding_key(x,res_pad))
json_input['attention'][0]['len'] = len_act
json_input['attention'][1]['len'] = len_res

In [173]:
len_res

43

In [174]:

data_add_prev

,caseid,concept:name,user,waiting_time,process_time,prev_acts,prev_res,curr_act,prev_acts_key_padding,prev_res_key_padding
80,10,10,7,0.000000,8.026497,"[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...","[27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 2...",[10],"[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru..."
81,10,12,13,10.090838,6.042633,"[10, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...","[7, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27...",[12],"[False, True, True, True, True, True, True, Tr...","[False, True, True, True, True, True, True, Tr..."
82,10,4,11,9.798183,7.390799,"[10, 12, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...","[7, 13, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27...",[4],"[False, False, True, True, True, True, True, T...","[False, False, True, True, True, True, True, T..."
83,10,18,15,11.226056,6.928538,"[10, 12, 4, 21, 21, 21, 21, 21, 21, 21, 21, 21...","[7, 13, 11, 27, 27, 27, 27, 27, 27, 27, 27, 27...",[18],"[False, False, False, True, True, True, True, ...","[False, False, False, True, True, True, True, ..."
84,10,11,15,7.273093,8.137688,"[10, 12, 4, 18, 21, 21, 21, 21, 21, 21, 21, 21...","[7, 13, 11, 15, 27, 27, 27, 27, 27, 27, 27, 27...",[11],"[False, False, False, False, True, True, True,...","[False, False, False, False, True, True, True,..."
...,...,...,...,...,...,...,...,...,...,...
5913,993,2,10,11.305914,5.707110,"[10, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...","[4, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27...",[2],"[False, True, True, True, True, True, True, Tr...","[False, True, True, True, True, True, True, Tr..."
5914,997,10,17,0.000000,7.878913,"[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...","[27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 2...",[10],"[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru..."
5915,997,2,20,10.525702,5.484797,"[10, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...","[17, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 2...",[2],"[False, True, True, True, True, True, True, Tr...","[False, True, True, True, True, True, True, Tr..."
5916,997,12,20,5.198497,5.198497,"[10, 2, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21...","[17, 20, 27, 27, 27, 27, 27, 27, 27, 27, 27, 2...",[12],"[False, False, True, True, True, True, True, T...","[False, False, True, True, True, True, True, T..."


In [55]:
# data = data_add_prev[[act_column, 'user_2', resource_column, 'waiting_time', 'process_time', 'prev_acts', 'prev_res','prev_acts_key_padding','prev_res_key_padding', 'curr_act']]
# data = data.reset_index(drop = True)
# data

,concept:name,user,waiting_time,process_time,prev_acts,prev_res,prev_acts_key_padding,prev_res_key_padding,curr_act
0,10,7,0.000000,8.026497,"[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...","[27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 2...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...",[10]
1,12,13,10.090838,6.042633,"[10, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...","[7, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27...","[False, True, True, True, True, True, True, Tr...","[False, True, True, True, True, True, True, Tr...",[12]
2,4,11,9.798183,7.390799,"[10, 12, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...","[7, 13, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27...","[False, False, True, True, True, True, True, T...","[False, False, True, True, True, True, True, T...",[4]
3,18,15,11.226056,6.928538,"[10, 12, 4, 21, 21, 21, 21, 21, 21, 21, 21, 21...","[7, 13, 11, 27, 27, 27, 27, 27, 27, 27, 27, 27...","[False, False, False, True, True, True, True, ...","[False, False, False, True, True, True, True, ...",[18]
4,11,15,7.273093,8.137688,"[10, 12, 4, 18, 21, 21, 21, 21, 21, 21, 21, 21...","[7, 13, 11, 15, 27, 27, 27, 27, 27, 27, 27, 27...","[False, False, False, False, True, True, True,...","[False, False, False, False, True, True, True,...",[11]
...,...,...,...,...,...,...,...,...,...
7212,2,10,11.305914,5.707110,"[10, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...","[4, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27...","[False, True, True, True, True, True, True, Tr...","[False, True, True, True, True, True, True, Tr...",[2]
7213,10,17,0.000000,7.878913,"[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...","[27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 2...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...",[10]
7214,2,20,10.525702,5.484797,"[10, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...","[17, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 2...","[False, True, True, True, True, True, True, Tr...","[False, True, True, True, True, True, True, Tr...",[2]
7215,12,20,5.198497,5.198497,"[10, 2, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21...","[17, 20, 27, 27, 27, 27, 27, 27, 27, 27, 27, 2...","[False, False, True, True, True, True, True, T...","[False, False, True, True, True, True, True, T...",[12]


In [175]:
data = data_add_prev[[act_column, resource_column, 'waiting_time', 'process_time', 'prev_acts', 'prev_res','prev_acts_key_padding','prev_res_key_padding', 'curr_act']]
data = data.reset_index(drop = True)
data

,concept:name,user,waiting_time,process_time,prev_acts,prev_res,prev_acts_key_padding,prev_res_key_padding,curr_act
0,10,7,0.000000,8.026497,"[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...","[27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 2...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...",[10]
1,12,13,10.090838,6.042633,"[10, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...","[7, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27...","[False, True, True, True, True, True, True, Tr...","[False, True, True, True, True, True, True, Tr...",[12]
2,4,11,9.798183,7.390799,"[10, 12, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...","[7, 13, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27...","[False, False, True, True, True, True, True, T...","[False, False, True, True, True, True, True, T...",[4]
3,18,15,11.226056,6.928538,"[10, 12, 4, 21, 21, 21, 21, 21, 21, 21, 21, 21...","[7, 13, 11, 27, 27, 27, 27, 27, 27, 27, 27, 27...","[False, False, False, True, True, True, True, ...","[False, False, False, True, True, True, True, ...",[18]
4,11,15,7.273093,8.137688,"[10, 12, 4, 18, 21, 21, 21, 21, 21, 21, 21, 21...","[7, 13, 11, 15, 27, 27, 27, 27, 27, 27, 27, 27...","[False, False, False, False, True, True, True,...","[False, False, False, False, True, True, True,...",[11]
...,...,...,...,...,...,...,...,...,...
7212,2,10,11.305914,5.707110,"[10, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...","[4, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27...","[False, True, True, True, True, True, True, Tr...","[False, True, True, True, True, True, True, Tr...",[2]
7213,10,17,0.000000,7.878913,"[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...","[27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 2...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...",[10]
7214,2,20,10.525702,5.484797,"[10, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...","[17, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 2...","[False, True, True, True, True, True, True, Tr...","[False, True, True, True, True, True, True, Tr...",[2]
7215,12,20,5.198497,5.198497,"[10, 2, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21...","[17, 20, 27, 27, 27, 27, 27, 27, 27, 27, 27, 2...","[False, False, True, True, True, True, True, T...","[False, False, True, True, True, True, True, T...",[12]


In [69]:
num_train = round(len(data)*0.99)
num_train

7145

In [362]:

from torch.utils.data import DataLoader
def infiniteloop(dataloader):
    while True:
        for _, y in enumerate(dataloader):
            yield y
            
# print(train_attention.tolist())
a = DataLoader(train_attention.tolist(),batch_size=5)
b = infiniteloop(a)
next(b)

[[tensor([14, 10, 10, 10, 10]),
  tensor([14, 14, 13, 13, 13]),
  tensor([14, 14, 14,  0,  0]),
  tensor([14, 14, 14, 14, 12]),
  tensor([14, 14, 14, 14, 14]),
  tensor([14, 14, 14, 14, 14]),
  tensor([14, 14, 14, 14, 14]),
  tensor([14, 14, 14, 14, 14]),
  tensor([14, 14, 14, 14, 14]),
  tensor([14, 14, 14, 14, 14]),
  tensor([14, 14, 14, 14, 14]),
  tensor([14, 14, 14, 14, 14])],
 [tensor([267, 234, 234, 234, 234]),
  tensor([267, 267, 266, 266, 266]),
  tensor([267, 267, 267, 124, 124]),
  tensor([267, 267, 267, 267, 235]),
  tensor([267, 267, 267, 267, 267]),
  tensor([267, 267, 267, 267, 267]),
  tensor([267, 267, 267, 267, 267]),
  tensor([267, 267, 267, 267, 267]),
  tensor([267, 267, 267, 267, 267]),
  tensor([267, 267, 267, 267, 267]),
  tensor([267, 267, 267, 267, 267]),
  tensor([267, 267, 267, 267, 267])],
 [tensor([10, 13,  0, 12,  7], dtype=torch.int32)]]

In [178]:
import json
with open("res_time_prev_0.json","w") as f:
    json.dump(json_input,f)

# with open('test.json') as json_file:
#     a = json.load(json_file)

train = np.array(data)
train_1 = train[:,0:-5].astype('float32')
train_attention = train[:,-5:]
train

test = np.array(data)
test_1 = test[:,0:-5].astype('float32')
test_attention = test[:,-5:]
test.shape
train_1


array_dict = {'train':train_1, 'test':test_1, 'train_attention': train_attention, 'test_attention' :test_attention}
np.savez('res_time_prev_0.npz', **array_dict)




In [177]:
res_list = json_input['attention'][1]['i2s']
res_dict = dict(zip(range(len(res_list)), res_list))
with open("res_time_prev_0_res_act.json","w") as f:
    json.dump(res_dict,f)

In [99]:
with open('res_time_prev_0.json') as json_file:
    a = json.load(json_file)
def transformer(x):
    data = [x]
    info = transformer_dis.meta[0]
    col_t = np.zeros([len(data), info['size']])
    idx = list(map(info['i2s'].index, data))
    col_t[np.arange(len(data)), idx] = 1
    data_t=col_t
    return data_t

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]])

In [105]:
data = np.array([[1,2]])
data[:,1][0]

2

In [74]:
a = np.load('res_time_prev_0.npz',allow_pickle=True)
a['train']

array([[10.       ,  7.       ,  0.       ,  8.026497 ],
       [12.       , 13.       , 10.090838 ,  6.042633 ],
       [ 4.       , 11.       ,  9.7981825,  7.3907986],
       ...,
       [16.       , 23.       , 10.862857 ,  4.7957907],
       [ 6.       , 23.       ,  0.       ,  0.       ],
       [14.       , 14.       ,  7.09091  ,  6.493754 ]], dtype=float32)

In [ ]:
#gen

In [179]:

import pandas as pd
import numpy as np
import torch
gen = pd.read_csv('new_technique_train_PurchasingExample.csv')
gen['activity'] = gen['act']
gen['act'] = gen['act'].map(lambda x: '_'.join(x.split()))

gen['act'] =gen['act'].map(lambda x:json_input['columns'][0]['i2s'].index(x) if x in json_input['columns'][0]['i2s'] else x)
gen
gen.to_csv('res_time_prev_0.csv', index=False)


In [131]:
# get the start time of each generated trace
ia_df = pd.read_csv(r'C:\Users\19wuj\PycharmProjects\CoDi\tabular_datasets\diffu\PurchasingExample_0.2\gen_PurchasingExample_1.csv')
ia_df
ia_list = []
for key, group in ia_df.groupby('caseid'):
    group['start_timestamp'] = group['start_timestamp'].map(lambda x: pd.to_datetime(x, format = '%Y-%m-%d  %H:%M:%S'))
    ia_list.append(min(group['start_timestamp']))
len(ia_list)

122

In [184]:
gen = pd.read_csv(r'C:\Users\19wuj\PycharmProjects\CoDi\tabular_datasets\diffu\PurchasingExample_0.2\gen_sample_7.csv')
gen = gen.set_index('Unnamed: 0')
gen['last'] = gen['last'].map(lambda x: [int(each) for each in x.strip('(').strip(')').split(',')] if x!='None' else x)
gen['next'] = gen['next'].map(lambda x: [int(each) for each in x.strip('(').strip(')').split(',')] if x!='None' else x)
gen

,caseid,act,index,last,next,if_para,start_timestamp,end_timestamp,resource,activity,res,wait,process
Unnamed: 0,,,,,,,,,,,,,
1,1,Start,1,None,[2],0,NaN,NaN,Start,Start,Start,0,0
2,1,10,2,[1],[3],0,NaN,NaN,Create Purchase Requisition,Create Purchase Requisition,Immanuel_Karagianni,0,1744
3,1,12,3,[2],[4],0,NaN,NaN,Create Request for Quotation,Create Request for Quotation,Christian_Francois,14511,905
4,1,4,4,[3],[5],0,NaN,NaN,Analyze Request for Quotation,Analyze Request for Quotation,Karel_de_Groot,726081,1410
5,1,1,5,[4],[6],0,NaN,NaN,Amend Request for Quotation,Amend Request for Quotation,Nico_Ojenbeer,3080,504
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2398,122,17,2398,[2397],[2399],0,NaN,NaN,Send Invoice,Send Invoice,Karen_Clarens,54779,0
2399,122,16,2399,[2398],[2400],0,NaN,NaN,Release Supplier's Invoice,Release Supplier's Invoice,Pedro_Alvares,52438,387
2400,122,6,2400,[2399],[2401],0,NaN,NaN,Authorize Supplier's Invoice payment,Authorize Supplier's Invoice payment,Karalda_Nimwada,0,0


In [185]:
g = pd.DataFrame()
for key, group in gen.groupby('caseid'):
    group = group.set_index('index')
    for i in range(len(group)):
        cur = group.iloc[i]
        if i == 0:
            group['start_timestamp'].iloc[i] = ia_list[int(key)-1]
            group['end_timestamp'].iloc[i] = group['start_timestamp'].iloc[i] + pd.Timedelta(seconds=0)
        else:
            if len(cur['last']) > 1:
                last_time = max(pd.to_datetime(group['end_timestamp'].loc[list(cur['last'])]))
                group['start_timestamp'].iloc[i] = last_time + pd.Timedelta(seconds=group['wait'].iloc[i])
            else:
                last_time = pd.to_datetime(group['end_timestamp'].loc[cur['last'][0]])
                group['start_timestamp'].iloc[i] = last_time + pd.Timedelta(seconds=group['wait'].iloc[i])
            group['end_timestamp'].iloc[i] = group['start_timestamp'].iloc[i] + pd.Timedelta(seconds=group['process'].iloc[i])
    g = pd.concat([g, group])

g['caseid'] = g['caseid'].map(lambda x: 'Case'+str(x))
g = g[['caseid', 'activity','start_timestamp', 'end_timestamp', 'res']]
g = g.rename(columns = {'activity':'task', 'res':'resource'})

In [186]:
g

,caseid,task,start_timestamp,end_timestamp,resource
index,,,,,
1,Case1,Start,2011-01-01 02:28:58,2011-01-01 02:28:58,Start
2,Case1,Create Purchase Requisition,2011-01-01 02:28:58,2011-01-01 02:58:02,Immanuel_Karagianni
3,Case1,Create Request for Quotation,2011-01-01 06:59:53,2011-01-01 07:14:58,Christian_Francois
4,Case1,Analyze Request for Quotation,2011-01-09 16:56:19,2011-01-09 17:19:49,Karel_de_Groot
5,Case1,Amend Request for Quotation,2011-01-09 18:11:09,2011-01-09 18:19:33,Nico_Ojenbeer
...,...,...,...,...,...
2398,Case122,Send Invoice,2011-01-22 19:27:41,2011-01-22 19:27:41,Karen_Clarens
2399,Case122,Release Supplier's Invoice,2011-01-23 10:01:39,2011-01-23 10:08:06,Pedro_Alvares
2400,Case122,Authorize Supplier's Invoice payment,2011-01-23 10:08:06,2011-01-23 10:08:06,Karalda_Nimwada


In [187]:
g.to_csv(f'gen_diffu_7.csv', index=False)

In [92]:





gen_res = []
gen_act = list(gen_act)
acts_prev = []
res_prev = []
for i in range(len(gen_act)):
    if gen_act[i] == 'Start':
        start_flag=i
        gen_res.append('Start')
    elif gen_act[i] == 'End':
        gen_res.append('End')
        # pass
    else:
        cur_act = [gen_act[i]]
        if i == start_flag+1:
            acts_prev = [23]
            res_prev = [13]

            acts_padding = [True]
            res_padding = [True]
        elif i == start_flag+2:
            acts_prev = []
            res_prev = []
            acts_prev.append(gen_act[i-1])
            res_prev.append(gen_res[i-1])

            acts_padding = [False for each in acts_prev]
            res_padding = [False for each in res_prev]
        else:
            acts_prev.append(gen_act[i - 1])
            res_prev.append(gen_res[i - 1])

            acts_padding = [False for each in acts_prev]
            res_padding = [False for each in res_prev]
        
        attention_tensor_list = [torch.tensor(acts_prev), torch.tensor(res_prev),
                                             torch.tensor(acts_padding),torch.tensor(res_padding),torch.tensor(cur_act) ]
        print(attention_tensor_list)
        gen_res.append(0)


[tensor([23]), tensor([13]), tensor([True]), tensor([True]), tensor([9])]
[tensor([9]), tensor([0]), tensor([False]), tensor([False]), tensor([6])]
[tensor([9, 6]), tensor([0, 0]), tensor([False, False]), tensor([False, False]), tensor([5])]
[tensor([23]), tensor([13]), tensor([True]), tensor([True]), tensor([9])]
[tensor([9]), tensor([0]), tensor([False]), tensor([False]), tensor([6])]
[tensor([9, 6]), tensor([0, 0]), tensor([False, False]), tensor([False, False]), tensor([5])]
[tensor([23]), tensor([13]), tensor([True]), tensor([True]), tensor([9])]
[tensor([9]), tensor([0]), tensor([False]), tensor([False]), tensor([6])]
[tensor([9, 6]), tensor([0, 0]), tensor([False, False]), tensor([False, False]), tensor([5])]
[tensor([23]), tensor([13]), tensor([True]), tensor([True]), tensor([9])]
[tensor([9]), tensor([0]), tensor([False]), tensor([False]), tensor([6])]
[tensor([9, 6]), tensor([0, 0]), tensor([False, False]), tensor([False, False]), tensor([1])]
[tensor([9, 6, 1]), tensor([0, 0

In [179]:
act_vocab = build_vocab(column='activity', min_freq=1)
act_vocab.get_stoi()

{'<bos>': 2,
 'Pending_Request_for_Reservation_Closure': 6,
 '<unk>': 0,
 'Request_created': 4,
 '<eos>': 3,
 '<pad>': 1,
 'Service_closure_Request_with_BO_responsibility': 5,
 'Pending_Request_for_acquittance_of_heirs': 12,
 'Request_completed_with_account_closure': 7,
 'Pending_Liquidation_Request': 8,
 'Service_closure_Request_with_network_responsibility': 9,
 'Authorization_Requested': 10,
 'Request_deleted': 14,
 'Evaluating_Request_(NO_registered_letter)': 11,
 'Network_Adjustment_Requested': 13,
 'Back-Office_Adjustment_Requested': 15,
 'Pending_Request_for_Network_Information': 16,
 'Evaluating_Request_(WITH_registered_letter)': 17}

In [177]:
class LoadActResDataset():
    def __init__(self, batch_size=2, min_freq=1):
        self.act_vocab = build_vocab(column='activity', min_freq=min_freq)
        self.res_vocab = build_vocab(column='user', min_freq=min_freq)
        self.specials = ['<unk>', '<pad>', '<bos>', '<eos>']
        self.PAD_IDX = self.act_vocab['<pad>']
        self.BOS_IDX = self.act_vocab['<bos>']
        self.EOS_IDX = self.act_vocab['<eos>']
        self.batch_size = batch_size
        
    def data_process(self):
        """
        将每一句话中的每一个词根据字典转换成索引的形式
        :param filepaths:
        :return:
        """
        raw_act_iter = data_add_prev['prev_acts']
        raw_res_iter = data_add_prev['prev_res']
        raw_curr_act_iter = data_add_prev['curr_act']
        data = []
        for (raw_act, raw_res, curr_act) in zip(raw_act_iter, raw_res_iter, raw_curr_act_iter):
            act_tensor_ = torch.tensor([self.act_vocab[token] for token in
                                       raw_act], dtype=torch.long)
            res_tensor_ = torch.tensor([self.res_vocab[token] for token in
                                       raw_res], dtype=torch.long)
            curr_act_tensor_ = torch.tensor([self.act_vocab[token] for token in
                                       curr_act], dtype=torch.long)
            data.append((act_tensor_, res_tensor_, curr_act_tensor_))
        return data



In [178]:
d = LoadActResDataset()
d.data_process()

[(tensor([], dtype=torch.int64), tensor([], dtype=torch.int64), tensor([4])),
 (tensor([4]), tensor([258]), tensor([9])),
 (tensor([4, 9]), tensor([258,   5]), tensor([10])),
 (tensor([ 4,  9, 10]), tensor([258,   5, 204]), tensor([5])),
 (tensor([ 4,  9, 10,  5]), tensor([258,   5, 204,   4]), tensor([6])),
 (tensor([ 4,  9, 10,  5,  6]),
  tensor([258,   5, 204,   4,   4]),
  tensor([8])),
 (tensor([ 4,  9, 10,  5,  6,  8]),
  tensor([258,   5, 204,   4,   4,   4]),
  tensor([7])),
 (tensor([], dtype=torch.int64), tensor([], dtype=torch.int64), tensor([4])),
 (tensor([4]), tensor([151]), tensor([11])),
 (tensor([ 4, 11]), tensor([151,  64]), tensor([9])),
 (tensor([ 4, 11,  9]), tensor([151,  64,  64]), tensor([5])),
 (tensor([ 4, 11,  9,  5]), tensor([151,  64,  64,   4]), tensor([6])),
 (tensor([ 4, 11,  9,  5,  6]),
  tensor([151,  64,  64,   4,   4]),
  tensor([8])),
 (tensor([ 4, 11,  9,  5,  6,  8]),
  tensor([151,  64,  64,   4,   4,   4]),
  tensor([7])),
 (tensor([], dtype=t

In [118]:
counter = Counter()
counter.update([each_ for each in data_add_prev['prev_acts'] for each_ in each])
counter

Counter({'Request_created': 1152,
         'Service_closure_Request_with_network_responsibility': 801,
         'Authorization_Requested': 387,
         'Service_closure_Request_with_BO_responsibility': 638,
         'Pending_Request_for_Reservation_Closure': 399,
         'Pending_Liquidation_Request': 215,
         'Evaluating_Request_(NO_registered_letter)': 405,
         'Pending_Request_for_Network_Information': 28,
         'Back-Office_Adjustment_Requested': 16,
         'Network_Adjustment_Requested': 87,
         'Pending_Request_for_acquittance_of_heirs': 18,
         'Evaluating_Request_(WITH_registered_letter)': 40})

In [406]:
import torch.nn.functional as F

In [419]:
a = F.softmax(torch.tensor([[float('-inf'),float('-inf')],[9,9]]), dim=-1)
a[a.isnan()]=0
a

tensor([[0.0000, 0.0000],
        [0.5000, 0.5000]])

In [70]:
counter = Counter()
# counter.update([each for each in data_add_prev['prev_acts']])


In [71]:
counter

Counter({'Request_created': 1152,
         'Service_closure_Request_with_network_responsibility': 801,
         'Authorization_Requested': 387,
         'Service_closure_Request_with_BO_responsibility': 638,
         'Pending_Request_for_Reservation_Closure': 399,
         'Pending_Liquidation_Request': 215,
         'Evaluating_Request_(NO_registered_letter)': 405,
         'Pending_Request_for_Network_Information': 28,
         'Back-Office_Adjustment_Requested': 16,
         'Network_Adjustment_Requested': 87,
         'Pending_Request_for_acquittance_of_heirs': 18,
         'Evaluating_Request_(WITH_registered_letter)': 40})